# Assigment 4

In [1]:
import math
import numpy as np
from numpy import matlib
import scipy.sparse as sp

import igl
import meshplot as mp

from math import sqrt

In [2]:
v, f = igl.read_triangle_mesh("data/irr4-cyl2.off")
tt, _ = igl.triangle_triangle_adjacency(f)

c = np.loadtxt("data/irr4-cyl2.constraints")
cf = c[:, 0].astype(np.int64)
c = c[:, 1:]

# 1. Tangent vector fields for scalar field design

In [3]:
def align_field(V, F, TT, hard_id, hard_value):
    assert(hard_id[0] > 0)
    assert(hard_id.shape[0] == hard_value.shape[0])

    
    # Edges
    e1 = V[F[:, 1], :] - V[F[:, 0], :]
    e2 = V[F[:, 2], :] - V[F[:, 0], :]

    # Compute the local reference systems for each face, T1, T2
    T1 = e1 / np.linalg.norm(e1, axis=1)[:,None]
        
    T2 =  np.cross(T1, np.cross(T1, e2))
    T2 /= np.linalg.norm(T2, axis=1)[:,None]
    
    # Project on the local frame
    c = np.zeros(hard_id.shape[0], dtype=np.complex)
    
    for ci in range(hard_id.shape[0]):
        f = hard_id[ci]
        v = hard_value[ci, :]
        
        c[ci] = np.dot(v, T1[f, :]) + np.dot(v, T2[f, :])* 1j
  
    # Arrays for the entries of the matrix
    data = []
    ii = []
    jj = []
    # Rhs of the system
    b = np.zeros(F.shape[0], dtype=np.complex)
    
    for f in range(F.shape[0]):
        for ei in range(3): # Loop over the edges
            
            # Look up the opposite face
            g = TT[f, ei]
            
            # If it is a boundary edge, it does not contribute to the energy
            # or avoid to count every edge twice
            if g == -1 or f > g:
                continue
                
            # Compute the complex representation of the common edge
            e  = V[F[f, (ei+1)%3], :] - V[F[f, ei], :]
            
            vef = np.array([np.dot(e, T1[f, :]), np.dot(e, T2[f, :])])
            vef /= np.linalg.norm(vef)
            ef = (vef[0] + vef[1]*1j)
            
            veg = np.array([np.dot(e, T1[g, :]), np.dot(e, T2[g, :])])
            veg /= np.linalg.norm(veg)
            eg = (veg[0] + veg[1]*1j)
            
            # Create the matrix Q and b
            if f in hard_id and g not in hard_id:
                data.append(1); ii.append(f); jj.append(f)
                data.append(eg * eg.conjugate()); ii.append(g); jj.append(g)
                
                b[f] = c[np.argwhere(hard_id == f)[0]]
                b[g] -= -ef.conjugate() * eg * c[np.argwhere(hard_id == f)[0]]
                
            elif f not in hard_id and g in hard_id:
                data.append(1); ii.append(g); jj.append(g)
                data.append(ef * ef.conjugate());  ii.append(f); jj.append(f)
                
                b[g] = c[np.argwhere(hard_id == g)[0]]
                b[f] -= -ef * eg.conjugate() * c[np.argwhere(hard_id == g)[0]]
                
            elif f not in hard_id and g not in hard_id:
                data.append(ef * ef.conjugate());  ii.append(f); jj.append(f)
                data.append(-ef * eg.conjugate()); ii.append(f); jj.append(g)
                data.append(-ef.conjugate() * eg); ii.append(g); jj.append(f)
                data.append(eg * eg.conjugate()); ii.append(g); jj.append(g) 
        
    # Lhs of the system
    Q = sp.coo_matrix((data, (ii, jj)), shape=(F.shape[0], F.shape[0])).asformat("csr")
    # Solve the linear system
    u = sp.linalg.spsolve(Q, b)
    R = T1 * u.real[:,None] + T2 * u.imag[:,None]

    return R

In [4]:
def plot_mesh_field(V, F, R, constrain_faces):
    # Highlight in red the constrained faces
    col = np.ones_like(F)
    col[constrain_faces, 1:] = 0
    
    # Scaling of the representative vectors
    avg = igl.avg_edge_length(V, F)/2

    #Plot from face barycenters
    B = igl.barycenter(V, F)

    p = mp.plot(V, F, c=col)
    p.add_lines(B, B + R * avg)
    
    return p

In [5]:
R = align_field(v, f, tt, cf, c)
plot_mesh_field(v, f, R, cf)

C:\Users\82778\AppData\Local\Temp\ipykernel_16560\1609457659.py:17: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  c = np.zeros(hard_id.shape[0], dtype=np.complex)
C:\Users\82778\AppData\Local\Temp\ipykernel_16560\1609457659.py:30: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  b = np.zeros(F.shape[0], dtype=np.complex)
D:

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# 2. Reconstructing a scalar field from a vector field

In [6]:
def scalar_field(V, F, u):
    A = sp.diags(np.tile(0.5 * igl.doublearea(V, F), 3))
    G = igl.grad(V, F)
    K = G.T @ A @ G
    b = -2 * G.T @ A @ u
    Lhs = K.T + K
    # Lhs[0, :] = 0
    # Lhs[:, 0] = 0
    # Lhs[0, 0] = 1
    Rhs = -b
    # Rhs[0] = 0
    s = sp.linalg.spsolve(Lhs, Rhs)
    g = G @ s
    g = np.reshape(g, f.shape, order='F')
    return s, g

In [7]:
def plot_scalar_field(V, F, s, g):
    # Scaling of the representative vectors
    avg = igl.avg_edge_length(V, F)/2

    #Plot from face barycenters
    B = igl.barycenter(V, F)

    p = mp.plot(V, F, c=s)
    p.add_lines(B, B + g * avg)
    
    return p

In [8]:
def plot_error_field(V, F, R, g):
    col = np.ones_like(f)
    # Scaling of the representative vectors
    avg = igl.avg_edge_length(V, F)/2

    #Plot from face barycenters
    B = igl.barycenter(V, F)

    p = mp.plot(V, F, c=col)
    p.add_lines(B, B + g * avg, shading={"line_color": "red"})
    p.add_lines(B, B + R * avg, shading={"line_color": "blue"})

    error = np.linalg.norm(R - g)
    print("Poisson Error: " + str(error))
    
    return p

In [9]:
s, g = scalar_field(v, f, R.T.flatten())
plot_scalar_field(v, f, s, g)
plot_error_field(v, f, R, g)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Poisson Error: 7.254251272979072


# 3. Harmonic and LSCM Parameterizations

In [10]:
v_camel, f_camel = igl.read_triangle_mesh("data/camel_head.off")

In [11]:
def harmonic(V, F):
    # Find the open boundary
    bnd = igl.boundary_loop(F)

    # Map the boundary to a circle, preserving edge proportions
    bnd_uv = igl.map_vertices_to_circle(V, bnd)

    # Harmonic parametrization for the internal vertices
    uv = igl.harmonic_weights(V, F, bnd, bnd_uv, 1)
    g = np.reshape(igl.grad(V, F) @ uv[:, 1], F.shape, order='F')
    return g, uv


In [12]:
def LSCM(V, F):
    # Fix two points on the boundary
    b = np.array([2, 1])

    bnd = igl.boundary_loop(F)
    b[0] = bnd[0]
    b[1] = bnd[int(bnd.size / 2)]

    bc = np.array([[0.0, 0.0], [1.0, 0.0]])

    # LSCM parametrization
    _, uv = igl.lscm(V, F, b, bc)

    return uv

In [13]:
def plot_parameterizations(V, F, uv_harmonic, uv_LSCM, g_harmonic):
    p = mp.subplot(V, F, s=[1, 4, 0], uv=uv_harmonic)
    mp.subplot(uv_harmonic, F, shading={"wireframe": True}, s=[1, 4, 1], data=p)

    avg = igl.avg_edge_length(V, F) / 2
    B = igl.barycenter(V, F)
    pp = mp.plot(V, F, uv=uv_harmonic, return_plot=True)
    pp.add_lines(B, B + g_harmonic * avg)
    
    mp.subplot(V, F, s=[1, 4, 2], uv=uv_LSCM, data=p)
    mp.subplot(uv_LSCM, F, shading={"wireframe": True}, s=[1, 4, 3], data=p)



In [14]:
g_harmonic, uv_harmonic = harmonic(v_camel, f_camel)
uv_LSCM = LSCM(v_camel, f_camel)
plot_parameterizations(v_camel, f_camel, uv_harmonic, uv_LSCM, g_harmonic)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.9967555…

# 4. Editing a parameterization with vector fields

In [15]:
g_irrcyl, uv_irrcyl = harmonic(v, f)
p = mp.plot(v, f, uv=uv_irrcyl)
uv_irrcyl[:, 1] = s

p = mp.plot(v, f, uv=uv_irrcyl)
avg = igl.avg_edge_length(v, f) / 2
B = igl.barycenter(v, f)
p.add_lines(B, B + g * avg, shading={"line_color": "gray"})
# p.add_lines(B, B + g_irrcyl * avg)
# g_u = np.reshape(igl.grad(v, f) @ uv_irrcyl[:, 0], f.shape, order='F')
# p.add_lines(B, B + g_u * avg)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

1